# `qubit_discovery` Advanced Features

In [ ]:
import numpy as np
import qubit_discovery as qd
import torch

: 

## Constructing custom metrics

While `qubit_discovery` offers a range of pre-made metrics to construct functions with, it's easy to create new ones for custom targets you have in mind. 

All loss functions in `qubit_discovery` have the signature
```
    def my_loss_function(circuit) -> Tuple(torch.Tensor, torch.Tensor)
```
They take in a single parameter (the circuit) and return a tuple of `(loss, metric)`. The `metric` is the target you want to track. The `loss` is a function of the metric which should be minimized, and can be linearly added to the overall loss function.

For instance, to optimize for a qubit with a particular operating frequency $f_{01}^*$, the metric you want to track is the current qubit frequency $f_{01}$ whereas the loss should be something like $\mathcal{L}_\text{freq} = |f_{01} - f_{01}^*|/f_{01}$.

In [1]:
def target_qubit_frequency_loss(circuit):
    TARGET_FREQ = 1.7   # GHz
    qubit_frequency = circuit.efreqs[1] - circuit.efreqs[0]
    loss = torch.abs(TARGET_FREQ - qubit_frequency)/TARGET_FREQ

    return loss, qubit_frequency

Then, we add it to the list of metrics that can be used to construct loss functions:

In [2]:
qd.losses.add_to_metrics('target_frequency', target_qubit_frequency_loss)

NameError: name 'qd' is not defined

During optimization, you can now use it with the `'target_frequency'` name:
```
calculate_loss_metrics(
    circuit,
    use_losses = ['target_frequency', ...],
    use_metrics = [...]
)
```

It's easy to make other loss functions. For instance, if you have a particular spectrum you wish to achieve, just modify the above function slightly.

For the metric, we track the qubit frequency $f_{01}$. (The metric value is simply provided as a convenient way to track data during the optimization procedure, and does not need to be used.)

In [ ]:
def target_spectrum_loss(circuit):
    TARGET_SPECTRUM = torch.tensor([0.014, 2.974, 0.194])

    circuit_spectrum = circuit.efreqs[1:4] - circuit.efreqs[0]
    loss = torch.mean(torch.abs(TARGET_SPECTRUM - circuit_spectrum)/TARGET_SPECTRUM[0])

    return loss, circuit_spectrum[0]

If you care about the circuit's spectrum at different operating points, that's also easy to do:

In [ ]:
def frequency_external_flux_loss(circuit):
    EXTERNAL_FLUXES = [0, 0.5]
    TARGET_FREQS = torch.tensor([4.5, 2.0])

    # Because we're changing the operating point, that could interfere with
    # other loss functions. Any loss function should always reset the circuit
    operating_point = circuit.loops[0].value()

    # Calculate qubit frequency at other flux points
    circuit_freqs = []
    for phi_ext in EXTERNAL_FLUXES:
        circuit.loops[0].set_flux(phi_ext)
        circuit.diag(2)
        circuit_freqs.append(circuit.efreqs[1] - circuit.efreqs[0])

    # Reset the circuit
    circuit.loops[0].set_flux(operating_point / 2 / np.pi)
    
    loss = torch.mean(torch.abs(TARGET_FREQS - circuit_freqs)/TARGET_FREQS[0])
    
    return loss, circuit_freqs[0]

Besides functions of the spectrum, SQcircuit also provides differentiable eigenvectors, decoherence and dephasing rates, coupling operators, and matrix elements. These can all be used to construct loss functions.

TODO: good examples of other loss functions?

## Reparameterization